In [13]:
import ifcopenshell
import csv

def get_storey(element):
    # Find the IfcBuildingStorey for the given element
    for rel in element.ContainedInStructure:
        if rel.is_a("IfcRelContainedInSpatialStructure") and rel.RelatingStructure.is_a("IfcBuildingStorey"):
            return rel.RelatingStructure
    return None

# Load the IFC file
ifc_file = ifcopenshell.open("Hus28_test.ifc")

# Search for the target storey by name
storey_name = "Plan 10"
target_storey = None

for storey in ifc_file.by_type("IfcBuildingStorey"):
    if storey.Name == storey_name:
        target_storey = storey
        break

if not target_storey:
    print(f"Storey with name '{storey_name}' was not found.")
else:
    # Find all IfcWindow elements
    ifc_windows = ifc_file.by_type("IfcWindow")

    # Initialize a dictionary to store windows and their connected rooms
    window_to_room = {}

    # Iterate over all IfcRelSpaceBoundary elements
    for rel in ifc_file.by_type("IfcRelSpaceBoundary"):
        # Check if the related element is a window
        if rel.RelatedBuildingElement in ifc_windows:
            window = rel.RelatedBuildingElement
            window_oid = window.id()
            
            # Check if the window is on the target storey
            window_storey = get_storey(window)
            if window_storey == target_storey:
                # Assign the room (IfcSpace) to the window
                room = rel.RelatingSpace
                if room:
                    room_name = room.Name
                    if window_oid not in window_to_room:
                        window_to_room[window_oid] = {"rooms": []}
                    window_to_room[window_oid]["rooms"].append(room_name)

    # Write the results to a CSV file
    with open('Output04_RoomsByWindows.csv', 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=';')
        for window_oid, data in window_to_room.items():
            row = [window_oid] + data["rooms"]
            csvwriter.writerow(row)

print("Results have been saved to Output02_RoomsByWindows.csv")


Results have been saved to Output02_RoomsByWindows.csv
